# Coupled-cluster doubles

In this notebook we set up a general solver for CCD given matrices $h_{p}^{q}$ and $u^{pq}_{rs}$, where the latter is _anti-symmetric_.

In [1]:
import pickle
import numpy as np
import time

We start by retrieving the pickled one-body and two-body matrix elements generated in the `matrix_elements.ipynb`-notebook.

In [2]:
with open("dat/h.pkl", "rb") as f:
    h = pickle.load(f)
with open("dat/u.pkl", "rb") as f:
    u = pickle.load(f)

## Iteration scheme

Using the equations (A7a and A7b) listed in "Ab initio quantum dynamics using coupled-cluster" by Simen Kvaal we can define iterations to find the amplitudes $\tau$. We define (note that there are no summing in these expressions)

\begin{align}
    f_{ij}^{ab} &= \frac{\partial}{\partial \lambda_{ab}^{ij}}\mathcal{E}_{H^{(1)}} \\
        &= -h_{c}^{a}\tau_{ij}^{bc}P(ab) + h_{i}^{k}\tau_{jk}^{ab}P(ij),
\end{align}

where $P(ij)$ is the anti-symmetrization operator permuting the two indices $i$, $j$ and subtracting the result. We start by assuming that $h$ is a diagonal matrix thus making a simplified expression for the right hand side of $f$.

\begin{align}
    f_{ij}^{ab} &= -h_{a}^{a}\tau_{ij}^{ba}P(ab) + h_{i}^{i}\tau_{ji}^{ab}P(ij) \\
        &= -h_{a}^{a}\tau_{ij}^{ba} + h_{b}^{b}\tau_{ij}^{ab}
            + h_{i}^{i}\tau_{ji}^{ab} - h_{j}^{j}\tau_{ij}^{ab} \\
        &= (h_{a}^{a} + h_{b}^{b} - h_{i}^{i} - h_{j}^{j})\tau_{ij}^{ab} \\
        &= -D_{ij}^{ab}\tau_{ij}^{ab},
\end{align}

where $D = D(h)$ given by

\begin{align}
    D_{ij}^{ab} = h_{i}^{i} + h_j^j - h_a^a - h_b^b.
\end{align}

The equations from A7b in Kvaal's article is given by

\begin{align}
    g_{ij}^{ab} &= \frac{\partial}{\partial\lambda_{ab}^{ij}}\mathcal{E}_{H^{(2)}} \\
        &= - \frac{\tau^{dc}_{jm} \tau^{ba}_{in}}{2} u^{mn}_{dc} P(ij)
            + \frac{\tau^{dc}_{ji} \tau^{ba}_{mn}}{4} u^{mn}_{dc}
            + \frac{\tau^{dc}_{ji} u^{ba}_{dc}}{2} \\
            &\qquad
            - \tau^{ac}_{im} \tau^{bd}_{jn} u^{mn}_{dc} P(ij)
            + \tau^{ac}_{im} u^{bm}_{jc} P(ab) P(ij)
            - \frac{\tau^{ac}_{ji} \tau^{bd}_{mn}}{2} u^{mn}_{dc} P(ab) \\
            &\qquad
            - \tau^{ac}_{ji} u^{bm}_{cm} P(ab)
            + \frac{\tau^{ba}_{mn} u^{mn}_{ji}}{2}
            + \tau^{ba}_{im} u^{mn}_{jn} P(ij) + u^{ba}_{ji}.
\end{align}

We define $g = g(u, \tau)$ and $f = f(h, \tau)$. We have that
\begin{align}
    f + g = 0 \iff -f = g.
\end{align}

We now define an iteration by introducing

\begin{align}
    \tau^{(k + 1)} = \frac{g(u, \tau^{(k)})}{D},
\end{align}

and start with the initial guess

\begin{align}
    \tau^{(0)} = \frac{u}{D}.
\end{align}

def compute_ccd_energy(h, u, tau, n):
    e_ref = np.einsum("ii->", h[:n, :n]) + 0.5*np.einsum("ijij->", u[:n, :n, :n, :n])
    #return e_ref + 0.25*np.einsum("cdkl, klcd ->", tau, u[:n, :n, n:, n:])
    return e_ref + 0.25*np.einsum("abij, abij->", u[n:, n:, :n, :n], tau)

In [3]:
def compute_ccd_energy(h, u, t, n):
    rhs = 0
    term = 1 * (0.5) * np.einsum("lklk -> ", u[:n, :n, :n, :n])
    rhs += term
    term = 1 * (0.25) * np.einsum("dclk, lkdc -> ", t, u[:n, :n, n:, n:])
    rhs += term
    term = 1 * np.einsum("kk -> ", h[:n, :n])
    rhs += term

    return rhs

def g(h, u, tau, n):
    rhs = np.zeros(tau.shape)

    term = -0.5*np.einsum("dcjm, bain, mndc -> abij", tau, tau, u[:n, :n, n:, n:])
    rhs += term - term.swapaxes(2, 3)

    rhs += 0.25*np.einsum("dcji, bamn, mndc -> abij", tau, tau, u[:n, :n, n:, n:])
    rhs += 0.5*np.einsum("dcji, badc -> abij", tau, u[n:, n:, n:, n:])

    term = -np.einsum("acim, bdjn, mndc -> abij", tau, tau, u[:n, :n, n:, n:])

    rhs += term - term.swapaxes(2, 3)

    term = np.einsum("acim, bmjc -> abij", tau, u[n:, :n, :n, n:])

    # The two expressions below produce the same result
    #rhs += term - term.swapaxes(0, 1) - term.swapaxes(2, 3) + term.swapaxes(0, 1).swapaxes(2, 3)
    rhs += (term - term.swapaxes(0, 1)) - (term - term.swapaxes(0, 1)).swapaxes(2, 3)

    term = -0.5*np.einsum("acji, bdmn, mndc -> abij", tau, tau, u[:n, :n, n:, n:])

    rhs += term - term.swapaxes(0, 1)

    term = -np.einsum("acji, bmcm -> abij", tau, u[n:, :n, n:, :n])

    rhs += term - term.swapaxes(0, 1)

    rhs += 0.5*np.einsum("bamn, mnji -> abij", tau, u[:n, :n, :n, :n])

    term = np.einsum("baim, mnjn -> abij", tau, u[:n, :n, :n, :n])

    rhs += term - term.swapaxes(2, 3)
    rhs += u[n:, n:, :n, :n]

    return rhs

In [4]:
def g(h, u, t, n):
    rhs = np.zeros(t.shape)
    term = 1 * (0.5) * np.einsum("balk, lkji -> abij", t, u[:n, :n, :n, :n])
    rhs += term
    term = 1 * (0.5) * np.einsum("dcji, badc -> abij", t, u[n:, n:, n:, n:])
    rhs += term
    
    one_dca = np.ones((len(h) - n, len(h) - n)) - np.eye(len(h) - n)
    term = 1 * np.einsum("ac, ac, bcji -> abij", one_dca, h[n:, n:], t)
    rhs += (term - term.swapaxes(0, 1))

    one_dki = np.ones((n, n)) - np.eye(n)
    term = 1 * (-1.0) * np.einsum("ki, ki, bajk -> abij", one_dki, h[:n, :n], t)
    rhs += (term - term.swapaxes(2, 3))

    term = 1 * (-1.0) * np.einsum("acji, bkck -> abij", t, u[n:, :n, n:, :n])
    rhs += (term - term.swapaxes(0, 1))
    term = 1 * (-1.0) * np.einsum("baik, lkjl -> abij", t, u[:n, :n, :n, :n])
    rhs += (term - term.swapaxes(2, 3))
    term = 1 * (0.25) * np.einsum("balk, dcji, lkdc -> abij", t, t, u[:n, :n, n:, n:])
    rhs += term
    term = 1 * np.einsum("acik, bdjl, lkdc -> abij", t, t, u[:n, :n, n:, n:])
    rhs += (term - term.swapaxes(2, 3))
    term = 1 * np.einsum("acik, bkjc -> abij", t, u[n:, :n, :n, n:])
    rhs += ((term - term.swapaxes(0, 1)) - (term - term.swapaxes(0, 1)).swapaxes(2, 3))
    term = 1 * (0.5) * np.einsum("bail, dcjk, lkdc -> abij", t, t, u[:n, :n, n:, n:])
    rhs += (term - term.swapaxes(2, 3))
    term = 1 * (-0.5) * np.einsum("acji, bdlk, lkdc -> abij", t, t, u[:n, :n, n:, n:])
    rhs += (term - term.swapaxes(0, 1))
    rhs += u[n:, n:, :n, :n]

    return rhs

In [5]:
def _diag_compute_amplitudes(h, u, n, tol=1e-4):
    # Here d_{ij}^{ab} = d[a][b][i][j]
    d = np.zeros((len(h) - n, len(h) - n, n, n))

    for a in range(len(h) - n):
        for b in range(len(h) - n):
            for i in range(n):
                for j in range(n):
                    d[a][b][i][j] = h[i][i] + h[j][j] - h[n + a][n + a] - h[n + b][n + b]

    tau = np.divide(u[n:, n:, :n, :n], d, where=(d != 0))

    e_ccd_prev = 0
    e_ccd = compute_ccd_energy(h, u, tau, n)
    diff = abs(e_ccd_prev - e_ccd)

    while diff > tol:
        tau = np.divide(g(h, u, tau, n), d)

        e_ccd_prev = e_ccd
        e_ccd = compute_ccd_energy(h, u, tau, n)

        diff = abs(e_ccd_prev - e_ccd)

    return e_ccd

In [6]:
def compute_amplitudes(h, u, n, diagonal_h=True):
    if not diagonal_h:
        raise NotImplementedError("This function currently only supports a diagonal h-matrix")
    else:
        return _diag_compute_amplitudes(h, u, n)

In [7]:
t1 = time.time()
e_ccd = compute_amplitudes(h, u, 2)
t2 = time.time()

print ("Time spent: {0}".format(t2 - t1))
print ("Energy: {0}".format(e_ccd))

Time spent: 0.13729643821716309
Energy: 1.7788892410077777


## Intermediates

Our current scheme for solving $g^{ab}_{ij}$ involves $\mathcal{O}(n^8)$ (expand on the exact cost) operations we wish to reduce this by pre-calculating some terms.

\begin{align}
    g_{ij}^{ab} &= \frac{\partial}{\partial\lambda_{ab}^{ij}}\mathcal{E}_{H^{(2)}} \\
        &= - \frac{\tau^{dc}_{jm} \tau^{ba}_{in}}{2} u^{mn}_{dc} P(ij)
            + \frac{\tau^{dc}_{ji} \tau^{ba}_{mn}}{4} u^{mn}_{dc}
            + \frac{\tau^{dc}_{ji} u^{ba}_{dc}}{2} \\
            &\qquad
            - \tau^{ac}_{im} \tau^{bd}_{jn} u^{mn}_{dc} P(ij)
            + \tau^{ac}_{im} u^{bm}_{jc} P(ab) P(ij)
            - \frac{\tau^{ac}_{ji} \tau^{bd}_{mn}}{2} u^{mn}_{dc} P(ab) \\
            &\qquad
            - \tau^{ac}_{ji} u^{bm}_{cm} P(ab)
            + \frac{\tau^{ba}_{mn} u^{mn}_{ji}}{2}
            + \tau^{ba}_{im} u^{mn}_{jn} P(ij) + u^{ba}_{ji}.
\end{align}

By collecting terms with common indices we get

\begin{gather}
    \frac{\tau^{dc}_{ji} \tau^{ba}_{mn}}{4} u^{mn}_{dc}
    + \frac{\tau^{dc}_{ji} u^{ba}_{dc}}{2}
    = \tau^{dc}_{ji}\left(
        \frac{1}{4}\tau^{ba}_{mn} u^{mn}_{dc}
        + \frac{1}{2} u^{ba}_{dc}
    \right)
    = \tau^{dc}_{ji}\chi^{ba}_{dc}, \\
    - \frac{\tau^{ac}_{ji} \tau^{bd}_{mn}}{2} u^{mn}_{dc} P(ab)
    - \tau^{ac}_{ji} u^{bm}_{cm} P(ab)
    = -\tau^{ac}_{ji}\left(
        \frac{1}{2}\tau^{bd}_{mn} u^{mn}_{dc}
        + u^{bm}_{cm}
    \right) P(ab)
    = -\tau^{ac}_{ji}\chi^{b}_{c} P(ab), \\
    - \frac{\tau^{dc}_{jm} \tau^{ba}_{in}}{2} u^{mn}_{dc} P(ij)
    = - \tau^{ba}_{in} \chi^n_j P(ij),
\end{gather}

where the $\chi$'s are _intermediates_. We can extract one more intermediate by writing

\begin{align}
    \tau^{ac}_{im} \tau^{bd}_{jn} u^{mn}_{dc}
    = \frac{1}{2} \tau^{ac}_{im} \tau^{bd}_{jn} u^{mn}_{dc} P(ab),
\end{align}

due to the antisymmetry of the coupled cluster amplitudes. This lets us write

\begin{align}
    - \tau^{ac}_{im} \tau^{bd}_{jn} u^{mn}_{dc} P(ij)
    + \tau^{ac}_{im} u^{bm}_{jc} P(ab) P(ij)
    &= -\frac{1}{2} \tau^{ac}_{im} \tau^{bd}_{jn} u^{mn}_{dc} P(ab) P(ij)
    + \tau^{ac}_{im} u^{bm}_{jc} P(ab) P(ij) \\
    &= \tau^{ac}_{im}\left(
        -\frac{1}{2} \tau^{bd}_{jn} u^{mn}_{dc}
        + u^{bm}_{jc}
    \right) P(ab) P(ij) \\
    &= \tau^{ac}_{im}\chi^{bm}_{jc} P(ab) P(ij).
\end{align}

In summary, this results in the following intermediates

\begin{align}
    \chi^{bm}_{jc} &= u^{bm}_{jc} - \frac{1}{2} \tau^{bd}_{jn} u^{mn}_{dc}, \\
    \chi^{b}_{c} &= \frac{1}{2}\tau^{bd}_{mn} u^{mn}_{dc} + u^{bm}_{cm}, \\
    \chi^n_j &= \frac{1}{2}\tau^{dc}_{jm} u^{mn}_{dc}, \\
    \chi^{ba}_{dc} &= \frac{1}{4}\tau^{ba}_{mn} u^{mn}_{dc} + \frac{1}{2} u^{ba}_{dc}.
\end{align}

## Updated two-body equation

Using the intermediates we can rewrite the equation for $g$ as

\begin{align}
    g^{ab}_{ij}
     &= - \tau^{ba}_{in} \chi^n_j P(ij)
     + \tau^{dc}_{ji}\chi^{ba}_{dc}
     + \tau^{ac}_{im}\chi^{bm}_{jc} P(ab) P(ij) \\
     &\qquad
     -\tau^{ac}_{ji}\chi^{b}_{c} P(ab)
     + \frac{\tau^{ba}_{mn} u^{mn}_{ji}}{2}
     + \tau^{ba}_{im} u^{mn}_{jn} P(ij) + u^{ba}_{ji}.
\end{align}

We now create a new function calculating the intermediates from

In [8]:
def g(h, u, tau, n):
    chi_bmjc = u[n:, :n, :n, n:] - 0.5*np.einsum("bdjn, mndc -> bmjc", tau, u[:n, :n, n:, n:])
    chi_bc = np.einsum("bmcm -> bc", u[n:, :n, n:, :n]) + 0.5*np.einsum("bdmn, mndc -> bc", tau, u[:n, :n, n:, n:])
    chi_nj = 0.5*np.einsum("dcjm, mndc -> nj", tau, u[:n, :n, n:, n:])
    chi_badc = 0.5*u[n:, n:, n:, n:] + 0.25*np.einsum("bamn, mndc -> badc", tau, u[:n, :n, n:, n:])

    term = -np.einsum("bain, nj -> abij", tau, chi_nj)
    rhs = term - term.swapaxes(2, 3)

    rhs += np.einsum("dcji, badc -> abij", tau, chi_badc)

    term = np.einsum("acim, bmjc -> abij", tau, chi_bmjc)

    term = term - term.swapaxes(2, 3)
    rhs += term - term.swapaxes(0, 1)

    term = -np.einsum("acji, bc -> abij", tau, chi_bc)
    rhs += term - term.swapaxes(0, 1)

    rhs += 0.5*np.einsum("bamn, mnji -> abij", tau, u[:n, :n, :n, :n])

    term = np.einsum("baim, mnjn -> abij", tau, u[:n, :n, :n, :n])
    rhs += term - term.swapaxes(2, 3)

    rhs += u[n:, n:, :n, :n]

    return rhs

In [9]:
t1 = time.time()
e_ccd = compute_amplitudes(h, u, 2)
t2 = time.time()

print ("Time spent: {0}".format(t2 - t1))
print ("Energy: {0}".format(e_ccd))

Time spent: 0.009082794189453125
Energy: 1.7788892410077777


## Further steps

1. Optimize code with respect to memory and speed.
2. Implement CCSD.
    * Compute the amplitude equations.
    * Intermediates factorization.
    * Write code for CCSD,
        - Try to re-use the CCD code as much as possible.
3. Implement the $\lambda$-equations (A9b).
    * Read chapter 1 in Helgaker.
4. Ground state calculations for DWQD.
    * Single particle basis.